In [19]:
import requests
import re
import json
from tqdm.auto import tqdm
from collections import defaultdict
import pickle 
import pandas as pd
import pymorphy2
import time

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

from sklearn.metrics.pairwise import linear_kernel

In [20]:
#скачиваем с api вакансии
sber = '3529'
page = 1
num_per_page = 100
ekaterinburg = 1261

url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={ekaterinburg}'
res = requests.get(url)
vacancies = res.json()
if vacancies is None:
        time.sleep(600)
        
# извлекаем требование и описание
all_vacancies = []
for i in tqdm(range(vacancies.get('pages'))):
    num_pages = vacancies.get('pages')
    vacancy_snippet = [el.get('snippet') for el in vacancies.get('items')]
    all_vacancies.extend(vacancy_snippet)
     
df = pd.DataFrame(all_vacancies)
df

  0%|          | 0/3 [00:00<?, ?it/s]

,requirement,responsibility
0,Высшее экономическое образование. Опыт работы ...,Проведение аудиторских проверок по направлению...
1,Специалист со средним профессиональным/неполны...,Осуществлять операции по обслуживанию физическ...
2,Опыт работы системным аналитиком от 2x лет (вы...,"Сбор, анализ, документирование и согласование ..."
3,Высшее образование. Опыт интеграции информацио...,Разработка «с нуля» новых сервисов и внедрение...
4,Специалист со средним профессиональным/неполны...,Осуществлять операции по обслуживанию физическ...
...,...,...
295,Знание и опыт разработки на SQL (желательно Or...,Развитие backend платформы (техстеков: Oracle ...
296,Специалист со средним профессиональным/неполны...,Осуществлять операции по обслуживанию физическ...
297,Знание финансового анализа предприятий. Опыт р...,"Анализ комплексных, нетиповых сделок по своей ..."
298,"Общие познания в области ИТ (сети, операционны...",Сопровождение розничных систем ДБО (интернет-б...


In [21]:
#предобрабатываем и добавляем преобразованный текст в новый столбец 'text_clean'
#удаляем пропуски
df.dropna(subset = ['responsibility'], inplace = True)

# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = '[^А-Яа-я0-9]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list
df['text_clean'] = remove_trash(df['responsibility'])

# функция удаления html-тэгов
def remove_trash(list): 
    pattern = r'<.*?>'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list
df['text_clean'] = remove_trash(df['responsibility'])

# функция лемматизации - приводит в 1 форму
morph = pymorphy2.MorphAnalyzer()
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:   #удаляет слова, которые меньше == 3 символа
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)
df['text_clean'] = df.apply(lemmatize,axis=1)

#удалить часто встречающиеся слова и стоп-слова
stop_russian = stopwords.words('russian')

#текст преобразуем в векторное представление
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=15000 )
text = text_transformer.fit_transform(df['responsibility'])

['Проведение аудиторских проверок по направлению Недвижимость эксплуатация строительство Наличное денежное обращение Оценка эффективности построения внутренних процессов и контрольных процедур в ', 'Осуществлять операции по обслуживанию физических лиц Проводить кассовые операции прием выдача обмен денег ', 'Сбор анализ документирование и согласование требований с заказчиками разработчиками и архитекторами Разработка документации к релизам Сопровождение разработки Сквозная коммуникация с ', 'Разработка с нуля новых сервисов и внедрение в промышленную эксплуатацию Доработка существующего функционала поддержка инфраструктуры Исправление дефектов Оценка сложности задач ', 'Осуществлять операции по обслуживанию физических лиц Проводить кассовые операции прием выдача обмен денег ', 'Ездить к клиентам по месту жительства или работы Вести переговоры с клиентами банка с целью возврата просроченной задолженности лично и ', 'Осуществлять операции по обслуживанию физических лиц Проводить кассовые 

In [22]:
#что ищем
text_to_find = 'Звонить существующим клиентам и предлагать продукты'

In [23]:
#создали матрицу по поиску
text_to_find_matrix = text_transformer.transform([text_to_find])

# расчет косинусного расстояния
cosine_similarities = linear_kernel(text, text_to_find_matrix).flatten()

# запишем все попарные результаты сравнений
df['cos_similarities'] = cosine_similarities
# и отсортируем по убыванию (т.к. cos(0)=1)
df = df.sort_values(by=['cos_similarities'], ascending=[0])

# Выведем 3 самых близких результата
for index, row in df[0:3].iterrows():
    print(row['responsibility'], row['cos_similarities'])

Продавать продукты банка и экосистемы физическим лицам. Помогать клиентам оформлять услуги на планшете, личном устройстве клиента или на устройстве самообслуживания... 0.19601720420881752
Продавать продукты банка и экосистемы физическим лицам. Помогать клиентам оформлять услуги на планшете, личном устройстве клиента или на устройстве самообслуживания... 0.19601720420881752
Продавать продукты банка и экосистемы физическим лицам. Помогать клиентам оформлять услуги на планшете, личном устройстве клиента или на устройстве самообслуживания... 0.19601720420881752
